In [1]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.0'
spark_version = 'spark-3.0.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backpor

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-02-07 00:10:54--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.1s    

2021-02-07 00:10:54 (6.55 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Lawn_and_Garden_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Lawn_and_Garden_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   32787517| RED72VWWCOS7S|B008HDQYLQ|     348668413|Garden Weasel Gar...| Lawn and Garden|          1|            2|          8|   N|                Y|            One Star|I don't hate the ...| 2015-08-31|
|         US|   16374060| RZHWQ208LTEPV|B005OBZBD6|     264704759|10 Foot Mc4 Solar...| Lawn and Garden|          5|    

### Create DataFrames to match tables

In [5]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame
df.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   32787517| RED72VWWCOS7S|B008HDQYLQ|     348668413|Garden Weasel Gar...| Lawn and Garden|          1|            2|          8|   N|                Y|            One Star|I don't hate the ...| 2015-08-31|
|         US|   16374060| RZHWQ208LTEPV|B005OBZBD6|     264704759|10 Foot Mc4 Solar...| Lawn and Garden|          5|    

In [6]:
# Create the customers_table DataFrame
customers_df = df.groupby("customer_id").agg({"customer_id":"count"}).withColumnRenamed("count(customer_id)", "customer_count").dropDuplicates()
customers_df.show(20)

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   46263557|             1|
|   44418648|             1|
|   34220092|             1|
|   10770444|             1|
|   12819130|             1|
|   24692214|             3|
|   51981844|             3|
|   43622169|             1|
|   43097449|             1|
|   45978717|             1|
|   20227781|             1|
|   11487525|             4|
|   31169607|             1|
|   22552080|             2|
|   15829398|             2|
|   46909180|             1|
|     198573|             3|
|   33383395|             1|
|    2934525|             3|
|    2333626|             1|
+-----------+--------------+
only showing top 20 rows



In [7]:
# Create the products_table DataFrame and drop duplicates. 
products_df = df.select(["product_id", "product_title"]).dropDuplicates(["product_id"])
products_df.show(20)

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|1602003920|Resurrection Eggs...|
|8805002585|All Weather Ivory...|
|B00002N67Q|Gilmour 10058050 ...|
|B00002N8FK|Heath Manufacturi...|
|B00002N8O2|Chapin Premier Co...|
|B00002NCGY|  Weed or Brier Hook|
|B00004R9TL|BLACK+DECKER AF-1...|
|B00004S1Q8|Ames True Temper ...|
|B00004S1VJ|Orbit Water Hose ...|
|B00004S25Y|Orbit Brass 3/4" ...|
|B00004SSSP|Stanley 6630 Chil...|
|B00004TR7P|Poulan 18-Inch Ba...|
|B00004U9WK|Oxo Good Grips 17...|
|B00004VWJF|Birkenstock Red S...|
|B00004WKMG|Fiskars 302519SS ...|
|B00004Y86B|POND WORKS PUMP 9...|
|B00004Z4MQ|Rumford Gardener ...|
|B00004Z4N7|Coleman LG20510E ...|
|B00004ZAW1|GardenSong 448-6 ...|
|B00006RK5I|Xantrex Technolog...|
+----------+--------------------+
only showing top 20 rows



In [8]:
# Create the review_id_table DataFrame. 
# Convert the 'review_date' column to a date datatype with to_date("review_date", 'yyyy-MM-dd').alias("review_date")
review_id_df = df.select(["review_id", "customer_id", "product_id", "product_parent", to_date("review_date", 'yyyy-MM-dd').alias("review_date")]).dropDuplicates()
review_id_df.show(20)

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
| R6J125A9S5H1G|   33399595|B00OMSJ9WG|     583972217| 2015-08-31|
|R1ECJZYO90Z4ZS|   34762385|B00W6X2XEK|     468018405| 2015-08-31|
|R1GJY3WAD0R9V2|   31465161|B00KRI58AC|     410592354| 2015-08-31|
|R3EYSXH0V4WDFO|   18880389|B00ENPZE5Y|     608386088| 2015-08-31|
|R1EDRCT29ANRKS|   14905761|B00ITD5YRE|     351861197| 2015-08-31|
| R664MGH4JLUBL|   38892733|B003599IUG|      82783000| 2015-08-31|
| RKNRB4OUJLKI3|   36137863|B000FG8B50|     751534736| 2015-08-31|
|R2NQGXPAIXE10Y|   47024610|B00HYKMJOO|     348092530| 2015-08-31|
|R38AMHQC7RQBJ3|   16941620|B002GJX0FU|     715372657| 2015-08-31|
|R2T8WUWL7JV0AD|     463440|B0039ZRL5K|     159984282| 2015-08-31|
|R37ZTLEDY3TYPD|   48757093|B00SY48CU4|     698384073| 2015-08-31|
|R2SGRK1WK0CC27|   44240776|B0010OHFCQ|     709352573| 2015-08

In [9]:
# Create the vine_table DataFrame
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"]).dropDuplicates()
vine_df.show(20)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3T6KGTZ6FOC9Q|          5|            0|          0|   N|                Y|
|R33GYOCUY6HTE7|          5|            0|          0|   N|                Y|
|R2SFQJ7U345LR2|          3|            2|          2|   N|                Y|
| R8Y006W6WVE9X|          3|            0|          0|   N|                Y|
| R9ZZ9JEZ60HDQ|          4|            3|          4|   N|                Y|
|R2EK5GP4UDNAXN|          1|            0|          1|   N|                Y|
| RF0D3N8BC2F8A|          5|            0|          0|   N|                Y|
|R2O8EA86BKD24G|          5|            1|          1|   N|                N|
|R3GR4NPXZ9M0UQ|          5|            0|          0|   N|                Y|
|R29ZFTTW99JPB7|          4|            1|          1|   N|     

### Connect to the AWS RDS instance and write each DataFrame to its table. 

In [10]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://<database>:5432/challenge"
config = {"user":"postgres", 
          "password": "<password>", 
          "driver":"org.postgresql.Driver"}

In [11]:
# Write review_id_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [12]:
# Write products_df to table in RDS
# about 3 min
products_df.write.jdbc(url=jdbc_url, table='products_table', mode=mode, properties=config)

In [13]:
# Write customers_df to table in RDS
# 5 min 14 s
customers_df.write.jdbc(url=jdbc_url, table='customers_table', mode=mode, properties=config)

In [14]:
# Write vine_df to table in RDS
# 11 minutes
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)